# Pipeline 1
This pipeline gets data from the first data bucket, processes it through five components: loading the data, cleaning the data, splitting the data, undersampling the train data and uploading it to the cloud. 

### Install (???)

In [1]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

  Obtaining dependency information for google-cloud-pipeline-components>2 from https://files.pythonhosted.org/packages/29/05/e2b13e7982506e6f63c2973c7a139fd8905dfd25ba5a7e03a0cb4541a76d/google_cloud_pipeline_components-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for google-cloud-aiplatform from https://files.pythonhosted.org/packages/f6/67/734b8c73b8e708a24301b8a0a072ddfe936816896d12af4884e4f7bbd3b0/google_cloud_aiplatform-1.35.0-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 205.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 2.3.0
google-cloud-aiplatform==1.35.0
google_cloud_pipeline_components version: 2.4.1


In [2]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from google_cloud_pipeline_components.v1.endpoint import (EndpointCreateOp,ModelDeployOp)
from google_cloud_pipeline_components.types import artifact_types

In [3]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "assignment-1-399115"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://temp_de2023_2056332"


### Component: Loading data

In [27]:
@dsl.component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def download_data(project_id: str, bucket: str, feature_dataset_name: str, label_dataset_name: str, 
                  feature_dataset: Output[Dataset], label_dataset: Output[Dataset]):
    '''Download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Get the client and bucket
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket)

     # Download the feature dataset
    blob1 = bucket.blob(feature_dataset_name)
    blob1.download_to_filename(feature_dataset.path + ".csv")
    
    # Download the label dataset
    blob2 = bucket.blob(label_dataset_name)
    blob2.download_to_filename(label_dataset.path + ".csv")
   
    logging.info('Download & Merge all Data complete!')

In [28]:
@dsl.component(
    packages_to_install=["pandas"],
    base_image="python:3.10.7-slim"
)

def merge_data(feature_dataset: Input[Dataset], label_dataset: Input[Dataset], merged_dataset: Output[Dataset]):
    import pandas as pd
    import logging 
    import sys
    # Download the datasets
    df_feature = pd.read_csv(feature_dataset.path + ".csv",index_col=None)
    df_label = pd.read_csv(label_dataset.path + ".csv",index_col=None)
    # merge them together
    merged_df = pd.merge(df_feature, df_label, on='Ind_ID')
    merged_df.to_csv(merged_dataset.path + ".csv", index=False, encoding='utf-8-sig')

### Component: cleaning data
Irrelevant columns are delted.

In [29]:
@dsl.component(
    packages_to_install=["pandas", "numpy"],
    base_image="python:3.10.7-slim"
)
def clean_data(merged_dataset: Input[Dataset], cleaned_dataset: Output[Dataset]):
    '''Deletes irrelevant columns and removes NA's'''
    import pandas as pd
    import logging
    import sys
    import numpy

    # Sets the logging config
    logging.basicConfig(stream=sys.stdout, level=logging.INFO) 

    #Loads the merged dataset in
    df = pd.read_csv(merged_dataset.path + ".csv", index_col=None)

    # Drops the columns: Birthday_count, Employed_days, Mobile_phone, Work_Phone, Phone, EMAIL_ID, Type_Occupation and Family_Members.
    cleandf = df.drop(columns= ["Birthday_count", "Employed_days", "Mobile_phone", "Work_Phone", "Phone", "EMAIL_ID", "Type_Occupation", "Family_Members"])

    # Save the cleaned dataset
    cleandf.to_csv(cleaned_dataset.path + ".csv", index=False, encoding='utf-8-sig')

### Component: Creating Train/Test split

In [30]:
@dsl.component(
    packages_to_install=["pandas", "scikit-learn"],
    base_image="python:3.10.7-slim"
)
def train_test_split(cleaned_dataset: Input[Dataset], dataset_train: Output[Dataset], dataset_test: Output[Dataset]):
    '''train_test_split'''
    import pandas as pd
    import logging 
    import sys
    from sklearn.model_selection import train_test_split as tts
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO) 
    
    # get the cleaned data from previous component
    alldata = pd.read_csv(cleaned_dataset.path + ".csv", index_col=None) 
    
    #create a train and test dataset
    train, test = tts(alldata, test_size=0.3)

    #create a train.csv file and a test.csv file
    train.to_csv(dataset_train.path + ".csv" , index=False, encoding='utf-8-sig')
    test.to_csv(dataset_test.path + ".csv" , index=False, encoding='utf-8-sig')


### Component: Undersampling

In [ ]:
# @dsl.component(
#     packages_to_install=["pandas", "scikit-learn","imbalanced-learn","imblearn", "numpy"],
#     base_image="python:3.10.7-slim"
# )
# def undersample_train(dataset_train: Input[Dataset], resampled_dataset_train: Output[Dataset]):
#     '''Apply undersampling to training set'''
#     import pandas as pd
#     import logging 
#     import sys
#     from imblearn.under_sampling import RandomUnderSampler
    
#     logging.basicConfig(stream=sys.stdout, level=logging.INFO) 
    
#     # get the train dataset from the previous component
#     train_data = pd.read_csv(dataset_train.path + "csv", index_col=None)

#     # random sampling
#     rus = RandomUnderSampler(random_state=42)
#     resampled_train_data = rus.fit_resample(train_data)

#     # create a file with the randomly undersampled data
#     resampled_train_data.to_csv(resampled_train_data.path + ".csv" , index=False, encoding='utf-8-sig')

### Component: upload the dataset to Google Cloud!

In [31]:
@dsl.component(
    packages_to_install=["google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def upload_dataset_to_gcs(project_id: str, resampled_train_data: Input[Dataset], dataset_test: Input[Dataset]):
    '''upload dataset to gsc'''
    from google.cloud import storage   
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)    
  
    # upload the model to GCS
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket)
    blob1 = bucket.blob(dataset_test)
    blob2 = bucket.blob(resampled_train_data)
   
    blob1.upload_from_filename(dataset_test.path + ".csv")   
    blob2.upload_from_filename(resampled_train_data.path + ".csv")
    logging.info("Upload to Google Cloud complete.")

### Pipeline: Call all components with the output from the previous component

In [ ]:
# pipeline component
@kfp.dsl.pipeline(
    name="fraud-predictor-data-cleaning-pipeline")
def pipeline(project_id: str, data_bucket: str, model_repo: str, dataset_feature_name: str, dataset_label_name: str):    
    
    di_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        feature_dataset_name= dataset_feature_name,
        label_dataset_name = dataset_label_name
    )
    
    merge_op = merge_data(feature_dataset = di_op.outputs["feature_dataset"],
                          label_dataset = di_op.outputs["label_dataset"])
                          # merged_dataset: Output[Dataset])
     
    clean_op = clean_data(merged_dataset =  merge_op.outputs["merged_dataset"]) 
                                            # cleaned_dataset: Output[Dataset]) 
                                            
    train_test_split_op = train_test_split(cleaned_dataset = clean_op.outputs["cleaned_dataset"])
                                           # dataset_train: Output[Dataset], dataset_test: Output[Dataset])
    
    #resampled_train_op = undersample_train(dataset_train = train_test_split_op.outputs["dataset_train"])
                                            # resampled_dataset_train: Output[Dataset]

    upload_dataset_to_gcs(resampled_train_data = train_test_split_op.outputs["dataset_train"], dataset_test = train_test_split_op.outputs["dataset_test"],
                         project_id = project_id)
 

In [ ]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='assignment1_datacleaning_pipeline.yaml')

In [35]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    location=REGION,
)

# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="assignment_1_datacleaning",
    enable_caching=False,
    template_path="assignment1_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    location=REGION,
    parameter_values={
        'project_id': PROJECT_ID, # makesure to use your project id 
        'data_bucket': 'data_de2023_2056332',  # the data_bucket name
        'model_repo':'models_de2023_2056332', # GCP model repo bucket name
        'dataset_feature_name' : 'Credit_card.csv', # the feature dataset name
        'dataset_label_name': 'Credit_card_label.csv' # the label dataset name
    }
)

job.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/553335810270/locations/us-central1/pipelineJobs/fraud-predictor-data-cleaning-pipeline-20231019125239
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/553335810270/locations/us-central1/pipelineJobs/fraud-predictor-data-cleaning-pipeline-20231019125239')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/fraud-predictor-data-cleaning-pipeline-20231019125239?project=553335810270
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/553335810270/locations/us-central1/pipelineJobs/fraud-predictor-data-cleaning-pipeline-20231019125239 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatf

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [undersample-train].; Job (project_id = assignment-1-399115, job_id = 2039386811582644224) is failed due to the above error.; Failed to handle the job: {project_number = 553335810270, job_id = 2039386811582644224}"
